In [1]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q datasets
!pip install -q evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install qq wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 7.2 MB/s eta 0:00:00


In [3]:
from peft import PeftModel
from transformers import AutoModelForSeq2SeqLM

base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = PeftModel.from_pretrained(base_model, "ngocnamk3er/flan-t5-text2sql-lora")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
      

In [6]:
from datasets import load_dataset

# Tải bộ dữ liệu từ Hugging Face
dataset = load_dataset("gretelai/synthetic_text_to_sql")

# Truy cập phần train và test
train_data = dataset['train']
test_data = dataset['test']

# In 2 mẫu đại diện dưới dạng JSON từ phần train và test
import json

# Lấy 2 mẫu đầu tiên từ phần train và test
train_sample = train_data[0]
test_sample = test_data[0]

# In dưới dạng JSON
print("Train Sample:", json.dumps(train_sample, indent=2))
print("Test Sample:", json.dumps(test_sample, indent=2))


README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Train Sample: {
  "id": 5097,
  "domain": "forestry",
  "domain_description": "Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.",
  "sql_complexity": "single join",
  "sql_complexity_description": "only one join (specify inner, outer, cross)",
  "sql_task_type": "analytics and reporting",
  "sql_task_type_description": "generating reports, dashboards, and analytical insights",
  "sql_prompt": "What is the total volume of timber sold by each salesperson, sorted by salesperson?",
  "sql_context": "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180

In [7]:
LEFT_INDEX = 0
RIGHT_INDEX = 5851
BATCH_SIZE = 16

In [8]:
references = [
    test_data[idx]['sql'] for idx in range(LEFT_INDEX, RIGHT_INDEX)
]

In [9]:
import re
import json
import torch
from time import time
from torch.utils.data import DataLoader

extracted_sqls_with_context = []
# Định nghĩa dataset từ test_data
class SQLDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, left, right):
        self.dataset = dataset.select(range(left, right)).to_dict()
        self.samples = [dict(zip(self.dataset.keys(), values)) for values in zip(*self.dataset.values())]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return "Convert to SQL: " + sample['sql_prompt'] + " Context: " + sample['sql_context']

# Bắt đầu đo thời gian
start_time = time()

# Tạo DataLoader
sql_dataset = SQLDataset(test_data, LEFT_INDEX, RIGHT_INDEX)
dataloader = DataLoader(sql_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Định nghĩa file output
filename = f'format_output_with_context_{LEFT_INDEX}_{RIGHT_INDEX}.jsonl'

total_samples = 0  # Đếm tổng số mẫu

with open(filename, 'w') as json_file:
    for batch_idx, batch_sql_prompts in enumerate(dataloader):
        batch_start_time = time()  # Đo thời gian cho từng batch

        tokenizer.padding_side = "left"
        inputs = tokenizer(batch_sql_prompts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        # Sinh output từ mô hình
        with torch.no_grad():
          generated_ids = model.generate(
              **inputs,
              max_new_tokens=512,
              pad_token_id=tokenizer.eos_token_id
          )

        # Giải mã output
        generated_sqls = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)


        for idx, generated_sql in enumerate(generated_sqls):
            generated_sql = generated_sql.strip()  # Loại bỏ khoảng trắng hai đầu
            generated_sql = generated_sql.split(';', 1)[0] + ';' if ';' in generated_sql else generated_sql  # Lấy đến dấu ';' đầu tiên
            total_samples += 1
            extracted_sqls_with_context.append(generated_sql)
            result = {'id': LEFT_INDEX + batch_idx * BATCH_SIZE + idx + 1, 'answer': generated_sql}
            json.dump(result, json_file)
            json_file.write("\n")



        batch_end_time = time()
        print(f"⏱️ Batch {batch_idx + 1} processed in {batch_end_time - batch_start_time:.2f} seconds.")

# Kết thúc đo thời gian
end_time = time()
elapsed_time = end_time - start_time

# In ra kết quả
print(f"✅ Data saved to {filename}")
print(f"⏳ Total time taken: {elapsed_time:.2f} seconds")
if total_samples > 0:
    print(f"⏱️ Average time per sample: {elapsed_time / total_samples:.4f} seconds")


⏱️ Batch 1 processed in 6.19 seconds.
⏱️ Batch 2 processed in 3.81 seconds.
⏱️ Batch 3 processed in 5.41 seconds.
⏱️ Batch 4 processed in 28.78 seconds.
⏱️ Batch 5 processed in 30.37 seconds.
⏱️ Batch 6 processed in 29.14 seconds.
⏱️ Batch 7 processed in 4.32 seconds.
⏱️ Batch 8 processed in 7.03 seconds.
⏱️ Batch 9 processed in 11.30 seconds.
⏱️ Batch 10 processed in 6.04 seconds.
⏱️ Batch 11 processed in 4.73 seconds.
⏱️ Batch 12 processed in 27.88 seconds.
⏱️ Batch 13 processed in 6.81 seconds.
⏱️ Batch 14 processed in 6.78 seconds.
⏱️ Batch 15 processed in 28.34 seconds.
⏱️ Batch 16 processed in 5.60 seconds.
⏱️ Batch 17 processed in 28.72 seconds.
⏱️ Batch 18 processed in 4.06 seconds.
⏱️ Batch 19 processed in 5.98 seconds.
⏱️ Batch 20 processed in 4.71 seconds.
⏱️ Batch 21 processed in 7.47 seconds.
⏱️ Batch 22 processed in 4.16 seconds.
⏱️ Batch 23 processed in 5.89 seconds.
⏱️ Batch 24 processed in 5.62 seconds.
⏱️ Batch 25 processed in 18.92 seconds.
⏱️ Batch 26 processed in 5

In [10]:
extracted_sqls_with_context

["SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');",
 "DELETE FROM rural_infrastructure WHERE country = 'Indonesia' AND completion_date  2010;",
 "SELECT COUNT(*) FROM Accidents WHERE launch_provider IN ('SpaceX', 'Blue Origin');",
 'SELECT MAX(quantity) FROM sales;',
 'SELECT SUM(budget) FROM Movies_Release_Year WHERE release_year  2010;',
 "INSERT INTO attorneys (attorney_id, attorney_name, attorney_email, attorney_phone) VALUES ('Oliver Martinez', 'oliver.martinez@lawfirm.com', 555-555-5678);",
 'SELECT plant_name, COUNT(*) as total_emissions FROM co2_emissions GROUP BY plant_name ORDER BY total_emissions DESC LIMIT 2;',
 "SELECT SUM(total_cost) FROM climate_communication WHERE location = 'Antarctica' AND start_date >= DATE_SUB(CURRENT_DATE, INTERVAL 1 YEAR) AND end_date  DATE_SUB(CURRENT_DATE, INTERVAL 1 YEAR);",
 'SELECT name, conservation_status FROM marine_species;',
 "SELECT AVG(publications) FROM researchers WHERE organization = 'N

In [11]:
for x in extracted_sqls_with_context:
  print(x)
  print("--------------")

SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');
--------------
DELETE FROM rural_infrastructure WHERE country = 'Indonesia' AND completion_date  2010;
--------------
SELECT COUNT(*) FROM Accidents WHERE launch_provider IN ('SpaceX', 'Blue Origin');
--------------
SELECT MAX(quantity) FROM sales;
--------------
SELECT SUM(budget) FROM Movies_Release_Year WHERE release_year  2010;
--------------
INSERT INTO attorneys (attorney_id, attorney_name, attorney_email, attorney_phone) VALUES ('Oliver Martinez', 'oliver.martinez@lawfirm.com', 555-555-5678);
--------------
SELECT plant_name, COUNT(*) as total_emissions FROM co2_emissions GROUP BY plant_name ORDER BY total_emissions DESC LIMIT 2;
--------------
SELECT SUM(total_cost) FROM climate_communication WHERE location = 'Antarctica' AND start_date >= DATE_SUB(CURRENT_DATE, INTERVAL 1 YEAR) AND end_date  DATE_SUB(CURRENT_DATE, INTERVAL 1 YEAR);
--------------
SELECT name, conservation_status FROM

In [12]:
references

["SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');",
 "DELETE FROM rural_infrastructure WHERE country = 'Indonesia' AND completion_date < '2010-01-01';",
 'SELECT launch_provider, COUNT(*) FROM Accidents GROUP BY launch_provider;',
 'SELECT MAX(quantity) FROM sales;',
 'SELECT SUM(budget) FROM Movies_Release_Year WHERE release_year < 2010;',
 "INSERT INTO attorneys (attorney_name, attorney_email, attorney_phone) VALUES ('Oliver Martinez', 'oliver.martinez@lawfirm.com', '555-555-5678');",
 'SELECT plant_name, SUM(co2_emission_per_ton_produced) AS total_co2_emission FROM co2_emissions GROUP BY plant_name ORDER BY total_co2_emission DESC LIMIT 2;',
 "SELECT SUM(total_cost) FROM climate_communication WHERE location = 'Antarctica' AND end_date >= '2010-12-31';",
 'SELECT name, conservation_status FROM marine_species;',
 "SELECT organization, AVG(publications) as avg_publications FROM researchers WHERE organization = 'National Geographic';",
 "SE

In [13]:
import evaluate

In [14]:
# Tính BLEU
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=extracted_sqls_with_context, references=references)

print(results)


{'bleu': 0.507682326324316, 'precisions': [0.7008027976819882, 0.5625887219149248, 0.4793407834700884, 0.4085876227509392], 'brevity_penalty': 0.9630812675295849, 'length_ratio': 0.9637462923638351, 'translation_length': 169283, 'reference_length': 175651}
